In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Reading the data

### Properties

In [ ]:
properties_raw_data = pd.read_csv('realproperties.csv', index_col = 0)

In [ ]:
properties_raw_data.dtypes

### Customers

In [ ]:
customers_raw_data = pd.read_csv('realcustomer.csv', index_col = 0)

In [ ]:
customers_raw_data.dtypes

# Data Cleaning and Preprocessing

## Properties

In [ ]:
# Creating a new variable that replicates the original data is crucial. 

properties = properties_raw_data.copy()

In [ ]:
# properties.describe() method allows us to examine statistical summaries of the data.
# However, by default, it only includes numeric columns. 
# To include all variables, we specify 'include=all' in the parameter. 

properties.describe(include='all')

In [ ]:
# Since 'ID' serves as an identifier rather than a numeric variable, it's best to change its data type.
# By converting it to a string, we ensure no numerical operations can be inadvertently performed on it. 
properties['id'] = properties['id'].astype(str)
# Let's examine the 'id' column to confirm our changes.
properties['id']

In [ ]:
# The same applies for the 'building' and 'property#' variables
properties['building'] = properties['building'].astype(str)
properties['property#'] = properties['property#'].astype(str)

properties.dtypes

In [ ]:
properties.describe(include='all')

In [ ]:
# Focusing on 'date of sale', we notice several issues:
# Among 267 entries, there are 44 unique ones, which is unusual and warrants a closer look.

properties['date_sale'].unique()

# Our investigation reveals multiple problems:

# 1. Some entries are '#NUM!', a placeholder for missing or erroneous values, likely carried over from an Excel file.

# 2. Every date is listed as the first of the month, regardless of the actual month. 
# This suggests that our data may only include information about the month of the transaction, rather than the precise date.

# 3. The 'date_sale' Series is currently classified as an 'object' type, 
# meaning the dates are treated as strings instead of actual dates.

In [ ]:
# Let's address these issues step by step.
# The first task is to replace the Excel-specific missing value indicator '#NUM!' with a format 
# that pandas can understand as a null value, i.e., 'pd.NA'.
properties['date_sale'] = np.where(properties['date_sale']=='#NUM!', pd.NA, properties['date_sale'])

properties['date_sale'].unique()

In [ ]:
# The next step is to convert the 'date_sale' column to a date type. 
properties['date_sale'] = pd.to_datetime(properties['date_sale'])
properties['date_sale']

In [ ]:
properties.describe(include="all", datetime_is_numeric=True)

In [ ]:
properties['type'].unique()

In [ ]:
# Now, let's turn our attention to the 'type' column. 
# Although it appears to be well-formatted, we'll standardize it further by converting all its entries to lowercase.
properties['type'] = properties['type'].str.lower()

In [ ]:
properties.head()

In [ ]:
properties.dtypes

# interestingly, price is also of object type and we definitely want it to be a float

In [ ]:
# The '$' symbol in the 'price' column is causing it to be treated as a string. 
# To fix this, we'll strip the '$' sign and create a new Series 'price$'.
properties['price$'] = properties['price'].str.strip('$')
properties.head()

In [ ]:
# Next, we intend to convert the 'price$' column to a float data type.
#properties['price$'] = properties['price$'].astype(float)

# Unfortunately, we encounter an error because of the thousands separator ',' still present in the data.

In [ ]:
# Since the .strip() method only removes characters from the beginning and end of a string, 
# The .replace() method is apt for this task, replacing all instances of ',' with an empty string.
properties['price$']= properties['price$'].replace(",","",regex=True) 

properties.head()

In [ ]:
# now we can carry on with casting price into a float
properties['price$'] = properties['price$'].astype(float)

In [ ]:
properties = properties.drop(['price'],axis=1)
properties.head()

In [ ]:
properties.dtypes

In [ ]:
properties['status'].unique()

# There don't appear to be any missing values.
# However, we notice some inconsistencies: the word 'sold' is surrounded by spaces and is capitalized.

In [ ]:
properties['status'] = properties['status'].str.strip()
properties['status'] = properties['status'].str.lower()

properties['status'].unique()

In [ ]:
properties = properties.rename(columns= {'status':'sold'})

In [ ]:
properties['sold'] = properties['sold'].map({'sold':1,'-':0})
properties['sold'].unique()

In [ ]:
properties.dtypes

In [ ]:
properties.isna().sum()

# From the initial glance, it seems there are no missing values, apart from the 'date_sale' column. 
# To be certain, you can verify this by inspecting each column with the .unique() method.

In [ ]:
# Final property dataset
properties

## Customers

In [ ]:
customers = customers_raw_data.copy()

In [ ]:
customers.columns.array

In [ ]:
# rename customerid in an appropriate way
customers = customers.rename(columns= {'\ufeffcustomerid':'customerid'})
customers.head()

In [ ]:
# Let's get an overview of our data.
customers.describe(include="all")

In [ ]:
customers.dtypes

In [ ]:
customers.isna().sum()

In [ ]:
customers = customers.rename(columns= {'entity':'individual'})
customers.head()

In [ ]:
customers['individual'].unique()

In [ ]:
# To make 'individual' a Boolean-like variable (at least conceptually), let's map 'Individual' to 1 and 'Company' to 0.
customers['individual'] = customers['individual'].map({'Individual':1,'Company':0})

In [ ]:
customers['sex'].unique()

# It appears there are three options: 'F', 'M', and an empty string. 
# We should map the empty string to NaN to indicate missing data.

In [ ]:
# We want to keep 'sex' as an 'object' variable, so we'll map 'F' and 'M' to '1' and '0' respectively. 
customers['sex'] = customers['sex'].map({'F':'1','M':'0', '':pd.NA})

In [ ]:
customers.isna().sum()

In [ ]:
customers['purpose'] = customers['purpose'].str.lower()
customers['source'] = customers['source'].str.lower()

customers.head()

In [ ]:
customers['mortgage'] = customers['mortgage'].map({'No':0,'Yes':1})

customers.head()

In [ ]:
# We want to create a new column 'full_name' that combines the 'name' and 'surname' of each customer.
customers['full_name'] = customers['name'] + " " + customers['surname']

customers.head()

In [ ]:
# Let's drop name and surname columns from the dataframe.
customers = customers.drop(['name', 'surname'], axis=1)

customers.head()

In [ ]:
customers['birth_date'] = pd.to_datetime(customers['birth_date'])

customers.head()

In [ ]:
customers.dtypes

In [ ]:
customers.isna().sum()

In [ ]:
customers.head()

## Combine the two tables

In [ ]:
properties.head()

In [ ]:
customers.head()

In [ ]:
# We will use the 'merge' function in pandas to combine the two dataframes.
# The 'customerid' column is designated as the common key on which the dataframes will be merged. 
# We opt for a left join, retaining all rows from the 'properties' dataframe and appending matching rows from the 'customers' dataframe. 
# If there's no match, the resultant dataframe will have 'NA' for the corresponding 'customers' dataframe columns.
pd.merge(properties, customers, on='customerid', how='left')

In [ ]:
# The merge did not work as intended.
properties.customerid.unique()

In [ ]:
# It's apparent that the two datasets differ - there are unexpected and unnecessary spaces.
customers.customerid.unique()

In [ ]:
# Let's trim the unnecessary white space from 'customerid' in the properties dataframe.
properties['customerid'] = properties['customerid'].str.strip()
customers['customerid'] = customers['customerid'].str.strip()

In [ ]:
# To ensure a successful merge, we must ensure that 'customerid' in the customers table has only unique values.
customers.count()

In [ ]:
properties['customerid'].nunique()

In [ ]:
# We've observed that there's an empty space in 'customerid' in the 'properties' dataframe.
# Let's replace it with 'NA'. This corresponds to properties that are not yet purchased.
properties['customerid'] = np.where(properties['customerid']=='', pd.NA, properties['customerid'])

In [ ]:
# Let's rename our merged dataframe to 'real_estate_data' for clarity.
real_estate_data = pd.merge(properties, customers, on='customerid', how='left')

In [ ]:
real_estate_data.head()

In [ ]:
real_estate_data.tail()

In [ ]:
real_estate_data.fillna(pd.NA)

In [ ]:
real_estate_data.dtypes

# Statistics

In [ ]:
# We'll assign the cleaned and preprocessed data to a new variable 'data'.
data = real_estate_data.copy()

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.describe(include="all", datetime_is_numeric = True)

## Breakdowns by building

We can examine:
1. Breakdown of totals by building (frequency distribution by building)
2. Breakdown of averages by building

In [ ]:
data['building'].unique()

In [ ]:
data.groupby('building').sum()

# it would be more sensible to select a subset of 'data' to aggregate.

### Breakdown of totals by building

In [ ]:
columns_of_interest = ['building', 'sold','mortgage']
totals_by_building = data[columns_of_interest].groupby("building").sum()
totals_by_building 

### Breakdown of averages by building

In [ ]:
columns_of_interest = ['building', 'area', 'price$', 'deal_satisfaction']
averages_by_building = data[columns_of_interest].groupby("building").mean()
averages_by_building

## Breakdowns by country and state

Country
1. Breakdown of totals by country (frequency distribution by country)
2. Breakdown of averages by country

State
3. Frequency distribution by state
4. Relative frequency by state
5. Cumulative frequency by state

### Breakdown of totals by country

In [ ]:
columns_of_interest = ['country', 'sold','mortgage']
totals_by_country = data[columns_of_interest].groupby("country").sum()
totals_by_country

In [ ]:
# Let's inspect the unique values in the 'country' column.
data['country'].unique()

# Clearly, there are duplicate entries for USA which needs to be addressed

In [ ]:
object_columns = data.select_dtypes(['object']).columns
object_columns

In [ ]:
# Using this, we select only the data from these columns and
# use the .apply() method to strip all white spaces from them simultaneously. 
data[object_columns] = data[object_columns].apply(lambda x: x.str.strip())

In [ ]:
totals_by_country = data[columns_of_interest].groupby("country").sum()
totals_by_country

### Breakdown of averages by country
Based on what you have seen before for the breakdowns by bulding and by state, please find the breakdown by country of the columns 'area', 'deal_satisfaction', and 'price$'

In [ ]:
# For the average computations, we use the following list of columns.
columns_of_interest = ['country', 'area', 'deal_satisfaction','price$']
averages_by_country = data[columns_of_interest].groupby("country").mean()
averages_by_country

### Frequency distribution by state

In [ ]:
columns_of_interest = ['state', 'sold','mortgage']
totals_by_state = data[columns_of_interest].groupby("state").sum()
totals_by_state

In [ ]:
totals_by_state.sold.sum()

In [ ]:
totals_by_country

In [ ]:
data['state'] = np.where(data['state']=='', pd.NA, data['state'])
data['state'] = np.where(data['country']!='USA', pd.NA, data['state'])

In [ ]:
totals_by_state = data[columns_of_interest].groupby("state").sum()
totals_by_state

In [ ]:
totals_by_state.sold.sum()

In [ ]:
# To focus on the relative and cumulative frequency of sales, we can refine our table by state.
columns_of_interest = ['state', 'sold']
sold_by_state = data[columns_of_interest].groupby("state").sum()
sold_by_state

In [ ]:
# To get a clearer picture, we can sort the values in descending order.
sold_by_state = sold_by_state.sort_values('sold', ascending=False)
sold_by_state

In [ ]:
# The term 'sold' may not be the best to describe frequency, so we can rename this column.
sold_by_state = sold_by_state.rename(columns={'sold':'frequency'})
sold_by_state

### Relative frequency distribution by state
1. Add a new column to the 'sold_by_state' data frame, called 'relative_frequency' which contains the relative frequency of the different states.
2. Add a new column to the 'sold_by_state' data frame, called 'cumulative_frequency' which contains the cumulative frequency of the different states. You can use your own tools to achieve this, or look up the .cumsum() method here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.cumsum.html

In [ ]:
# The relative frequency can be computed by dividing the frequency of each state by the total frequency.
sold_by_state['relative_frequency'] = sold_by_state['frequency']/sold_by_state['frequency'].sum()
sold_by_state

In [ ]:
# Cumulative frequency can be obtained using the 'cumsum()' function in pandas.
sold_by_state['cumulative_frequency'] = sold_by_state['relative_frequency'].cumsum()
sold_by_state

# Data Analysis

## Customers Age

In [ ]:
data.head()

In [ ]:
# Let's calculate the age of our buyers at the time of purchase.
# This can be done by subtracting the birth date from the sale date.
data['age_at_purchase'] = data['date_sale'] - data['birth_date']
data['age_at_purchase']

In [ ]:
# timedelta has different attributes than normal dates, which you can explore in the docs
# for our purposes, we want the age in integers, so we can work with it
# to access the number of days stored inside, we can use the attribute 'days'

#data['age_at_purchase'][0].days
type(data['age_at_purchase'][0].days)

In [ ]:

# It's important to note that the 'age_at_purchase' field we've just created is of timedelta datatype.
# A timedelta object represents a duration, the difference between two dates or times.
# For our analysis, we need the age in integers or floats for better manipulation.
# To convert it, we use the 'days' attribute to extract the number of days, as timedelta measures duration in days.

data['age_at_purchase'] = data['age_at_purchase'].apply(lambda x: x.days)
data['age_at_purchase']

# Notice that the datatype is float64, which may be due to how pandas internally manages timedelta objects.

In [ ]:
# To obtain the age in years at the time of purchase, we can divide the 'age_at_purchase' field by 365.
data['age_at_purchase'] = data['age_at_purchase']/365
data['age_at_purchase']

In [ ]:
# Since age is typically represented in whole numbers, we can round down the values using np.floor().
data['age_at_purchase_rounded'] = data['age_at_purchase'].apply(lambda x: np.floor(x))
data['age_at_purchase_rounded']

### Create age intervals

In [ ]:
# To divide age into different intervals, we can use the 'cut' method from pandas.
# This function segments and sorts the data values into bins we specify.
# Here, we're dividing the 'age_at_purchase' into 10 bins and setting the decimal precision to 0.
data['age_interval'] = pd.cut(data['age_at_purchase'], bins = 10, precision = 0)
data['age_interval']

### Breakdown by age interval

In [ ]:
columns_of_interest = ['age_interval', 'sold']
sold_by_age = data[columns_of_interest].groupby("age_interval").sum()
sold_by_age

## Analysis of the price of properties

### Price interval

In [ ]:
data['price_interval'] = pd.cut(data['price$'], bins=10)
data['price_interval']

### Total number of properties

In [ ]:
columns_of_interest = ['price_interval', 'sold']
all_properties_by_price = data[columns_of_interest].groupby("price_interval").count()

all_properties_by_price = all_properties_by_price.rename(columns={'sold':'count'})
all_properties_by_price

### Total number of sold properties

In [ ]:
columns_of_interest = ['price_interval', 'sold']
sold_properties_by_price = data[columns_of_interest].groupby("price_interval").sum()
sold_properties_by_price

### Total number of not sold properties

In [ ]:
# To identify properties that remain unsold, we can subtract the sold properties from the total count.
all_properties_by_price['not_sold'] = all_properties_by_price['count'] - sold_properties_by_price['sold']
all_properties_by_price['sold'] = sold_properties_by_price['sold']
all_properties_by_price

## Relationship between age and price

In [ ]:
data

### Filter out only the sold apartments

In [ ]:
data_sold = data[data['sold']==1]

In [ ]:
# Further, let's exclude any company data, focusing only on individual sales.
data_sold = data_sold[data_sold['individual']==1]

In [ ]:
data_sold[['age_at_purchase','price$']]

### Covariance of age and price

In [ ]:
np.cov(data_sold['age_at_purchase'], data_sold['price$'])

### Correlation of age and price

In [ ]:
np.corrcoef(data_sold['age_at_purchase'], data_sold['price$'])

In [ ]:
# As an alternative, we can use the correlation method provided by pandas, which gives the same result.
data_sold_no_na[['age_at_purchase','price$']].corr()

# Data visualization

## Deal Satisfaction Across Countries (Bar Chart)

1. To plot the deal satisfaction by country, we must first obtain the relevant data. There needs to be a breakdown of deal satisfaction by country. If you remember, we have done that ealier.
2. With the data we can create a bar chart with the following parameters: 
    - white background with a grid
    - size of the figure (12,6)
    - fitting color which is not the default one
    - rotated x ticks, with a font size of 13
    - y ticks, with a font size of 13
    - y label for the deal satisfaction (no need for an x label)
    - remove the top and the right border of the chart
    - save the newly created figure as a .png on your computer

We will maintain these visualization parameters for all subsequent charts in order to ensure consistency and comparability across the different visualizations.

In [ ]:
# we have already calculate the averages by country so we can take advantage of that
averages_by_country

In [ ]:
sns.set_style("whitegrid") 

plt.figure(figsize = (12, 6)) 

plt.bar(x = averages_by_country.index,height = averages_by_country['deal_satisfaction'],color = "#108A99") 

plt.xticks(rotation = 45, fontsize = 13) 
plt.yticks(fontsize = 13) 
plt.title("Deal Satisfaction by Country", fontsize = 18, fontweight = "bold") 
plt.ylabel("Deal Satisfaction", fontsize = 13 ) 

sns.despine() # removes the top and right border of our graph

plt.savefig("deal_satisfaction_by_country_bar_chart.png") 
plt.show()

##  Customer Age Distribution (Histogram) (Histogram)

1. Тo visualize the distribution of customer ages at the time of purchase, we'll generate a histogram. The necessary data for this visualization is already included in the data variable.
2. Histogram construction: We use the <strong>data</strong> variable to create a histogram that represents the age distribution at purchase.The histogram should incorporate the following parameters:
    - white background with a grid
    - size of the figure (12,6)
    - 10 bins for the different categories
    - fitting color which is not the default one
    - appropriate x and y labels
    - remove the top and the right border of the chart
    - save the newly created figure as a .png on your computer

In [ ]:
data.head()

In [ ]:
sns.set_style("whitegrid") 

plt.figure(figsize = (12, 6)) 
plt.hist(data['age_at_purchase'],bins = 10,color = "#108A99")
plt.title("Age Distribution", fontsize = 18, weight = "bold")
plt.xlabel("Age", fontsize=13)
plt.ylabel("Number of Purchases", fontsize=13)

sns.despine()
plt.savefig("age_distribution_histogram.png") 
plt.show()

## Total Sales per Year (Line chart)

In [ ]:
data.head()

In [ ]:
# We need to create a line chart detailing total sales per year.
data['date_sale'][0].year

In [ ]:
# We do that for the whole series and save the year in a separate column
data['year_sale'] = data['date_sale'].apply(lambda x: x.year)
data['year_sale']

In [ ]:
# We need to convert the 'year_sale' variable type from float to int as fractional years don't accurately represent our data.
# To modify the data type, we employ the 'astype' method. Since this function cannot handle missing values, we initially replace all NAs with zeros.
data['year_sale'] = data['year_sale'].fillna(0).astype(int)

# We also need to convert all zeros back to NA, restoring our dataset's missing value representation.
data['year_sale'] = data['year_sale'].replace({0:pd.NA})

In [ ]:
# Next, we formulate the desired sales breakdown by year.
columns_of_interest = ['year_sale','price$']
revenue_per_year = data[columns_of_interest].groupby('year_sale').sum()

revenue_per_year = revenue_per_year.rename(columns={'price$':'revenue$'})
revenue_per_year

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize = (12, 6))

plt.plot(revenue_per_year['revenue$'],color='#108A99',linewidth=3)

plt.title("Total Revenue per Year (2004-2010)", fontsize = 18, fontweight = "bold")
plt.ylabel("Revenue $", fontsize = 13)
plt.xticks(fontsize = 13) 
plt.yticks(fontsize = 13) 

sns.despine() # We'll remove the top and right borders of the chart for a cleaner look.
plt.savefig("total_revenue_per_year_in_M_line_chart.png")
plt.show() 

In [ ]:
# Several issues detract from the effectiveness of our graph:
# 1. Revenue figures are displayed in scientific notation, which is not readily interpretable.
# 2. The year 2009 lacks its own tick mark, due to its absence from the revenue_per_year dataframe.

# Currently, revenue is represented in dollars, but the figures are large and overwhelming.
# For a cleaner, more comprehensible visualization, we should present revenue in thousands or millions.
revenue_per_year['revenue$inM'] = revenue_per_year['revenue$'] / 1000000

In [ ]:
revenue_per_year

In [ ]:
# To improve our data representation, we'll add a new data point for the year 2009.
# This year will display 0 revenue, reflecting the actual situation.
# This inclusion aids in presenting a more accurate reality.
revenue_per_year_adj = revenue_per_year.copy() 

revenue_per_year_adj = revenue_per_year_adj.append({'revenue$': 0,'revenue$inM': 0},ignore_index=True)

revenue_per_year_adj.index = ['2004','2005','2006','2007','2008','2010','2009']
# Reordering the dataframe to chronologically arrange the years.
revenue_per_year_adj = revenue_per_year_adj.loc[['2004','2005','2006','2007','2008','2009','2010']]
revenue_per_year_adj

In [ ]:
sns.set_style("whitegrid") 

plt.figure(figsize = (12, 6)) 

plt.plot(revenue_per_year_adj['revenue$inM'],color='#108A99',linewidth=3)

plt.title("Total Revenue per Year (2004-2010)", fontsize = 18, fontweight = "bold")
plt.ylabel("Revenue $ in Millions", fontsize = 14)
plt.xticks(fontsize = 13) 
plt.yticks(fontsize = 13) 

sns.despine() 
plt.savefig("total_revenue_per_year_in_M_line_chart.png") 
plt.show() # Display the chart.

## Yearly Sales Distribution Across Buildings (Stacked Area Chart)
We're going to create a stacked area chart that displays the distribution of yearly sales across various buildings. Follow these step-by-step instructions:
1. <strong>Initialize a checkpoint dataframe:</strong> Establish a new variable named <strong>data_stacked_area</strong> which will be our checkpoint for further computations.
2.  <strong>Formulate building indicators:</strong> Construct indicator (or dummy) variables rooted in <strong>building</strong>. These can be stored directly in data_stacked_area. An alternate approach involves two steps: creating a separate <strong>building_dummies</strong> variable, and then appending it to <strong>data_stacked_area</strong>.
3. <strong>Redefine column names:</strong> Amend the names of the dummy variables to be more descriptive. Simple labels such as 'building1', 'building2', etc. should work well.
4. <strong>Exclude unsold properties:</strong> Remove all properties from the dataset that haven't been sold.
5. <strong>Generate a yearly breakdown:</strong> Develop a breakdown by year for the 5 building dummy variables. This will yield a yearly sales distribution per building.
6. <strong>Create a stacked area chart:</strong> With the data prepared, it's time to create the stacked area chart: 
    - Background: Opt for a white background with a grid.
    - Figure size: Set the dimensions as (12,6).
    - Color scheme: Use a 5-color scheme, with colors that distinctly represent the 5 buildings. 
    - Edgecolor: This should be none.
    - Legend: Include a legend that maps the colors to their corresponding labels.
    - Axes labels: Assign an x label and y label, both with a font size of 13.
    - Chart borders: Discard the top and right borders of the chart.
    - Save the figure: Store the final chart as a .png file on your local machine

One point to keep in mind, as we noted during the Line Chart creation, the year 2009 isn't present in the data frame we are plotting. Hence, its corresponding value (0) isn't marked. To address this, generate a new stacked area chart that includes the year 2009 as an observation.

In [ ]:
data_stacked_area = data.copy()

In [ ]:
building_dummies = pd.get_dummies(data_stacked_area['building'])
building_dummies.head()

In [ ]:
data_stacked_area = pd.concat([data_stacked_area, building_dummies], axis=1)
data_stacked_area.head()

In [ ]:
data_stacked_area = data_stacked_area.rename(columns={'1':'building1','2':'building2','3':'building3','4':'building4','5':'building5'})
data_stacked_area.head()

In [ ]:
# Finally, let's filter out only the sold apartments, as they are the only ones of interest.
data_stacked_area = data_stacked_area[data_stacked_area['sold'] == 1]

In [ ]:
columns_of_interest = ['year_sale','building1','building2','building3','building4','building5']
stacked_area = data_stacked_area[columns_of_interest].groupby('year_sale').sum()
stacked_area

In [ ]:
colors = ["#264653", "#2A9D8F", "E9C46A",'F4A261','E76F51']
labels = ['Building 1','Building 2','Building 3','Building 4','Building 5',]
sns.set_style("whitegrid")
plt.figure(figsize = (12, 6))

plt.stackplot(stacked_area.index, stacked_area['building1'],stacked_area['building2'],stacked_area['building3'],stacked_area['building4'],
              stacked_area['building5'],colors = colors,edgecolor = 'none')

plt.xticks(stacked_area.index, rotation = 45) 
plt.legend(labels = labels, loc = "upper left") 
plt.ylabel("Number of Sales", fontsize = 13)
plt.xticks(fontsize = 13)
plt.yticks(fontsize = 13)

plt.title("Total Number of Sales per Year by Building", fontsize = 18)
sns.despine()
plt.savefig("total_sales_per_year_per_building_stacked_area_chart.png")
plt.show()

In [ ]:
stacked_area_adj = stacked_area.copy() 
stacked_area_adj = stacked_area_adj.append({'building1': 0, 
                                     "building2": 0, 
                                     "building3": 0,
                                     "building4": 0,
                                     "building5": 0},ignore_index=True)

# Adjust the index to include 2009.
stacked_area_adj.index = ['2004','2005','2006','2007','2008','2010','2009']
stacked_area_adj = stacked_area_adj.loc[['2004','2005','2006','2007','2008','2009','2010']]

######################################################################################################

colors = ["#264653", "#2A9D8F", "E9C46A",'F4A261','E76F51']
labels = ['Building 1','Building 2','Building 3','Building 4','Building 5',]
sns.set_style("whitegrid")
plt.figure(figsize = (12, 6))

plt.stackplot(stacked_area_adj.index, stacked_area_adj['building1'],stacked_area_adj['building2'],stacked_area_adj['building3'],
              stacked_area_adj['building4'],stacked_area_adj['building5'],colors = colors,edgecolor = 'none')

plt.xticks(stacked_area_adj.index, rotation = 45) 
plt.legend(labels = labels, loc = "upper left") 
plt.ylabel("Number of Sales", fontsize = 13)
plt.xticks(fontsize = 13) 
plt.yticks(fontsize = 13) 

plt.title("Total Number of Sales per Year by Building", fontsize = 18)
sns.despine()
plt.savefig("total_sales_per_year_per_building_stacked_area_chart_v2.png")
plt.show()